In [3]:
import pandas as pd

In [4]:
df=pd.read_csv("gemstone.csv")

In [5]:
df=df.drop(labels=['id'],axis=1)

In [6]:
## Independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [7]:
Y

,price
0,499
1,984
2,6289
3,1082
4,779
...,...
26962,5408
26963,1114
26964,1656
26965,682


In [8]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [9]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']


In [10]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [12]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [13]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [14]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.561674,0.318936,-0.651005,0.704838,0.692859,0.723432,0.983583,0.226283,0.576148
1,-0.604087,-0.397691,1.589878,-0.533052,-0.486169,-0.533329,-0.812066,-0.945547,-0.639724
2,-0.520818,-0.254365,-1.099182,-0.409263,-0.376688,-0.410385,0.983583,0.812198,-1.247659
3,-0.999613,0.247274,-1.099182,-1.178523,-1.134635,-1.107067,0.983583,0.812198,0.576148
4,0.228599,0.963901,-0.651005,0.413049,0.330729,0.477544,-0.812066,0.812198,-0.031788


In [15]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [16]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [17]:
regression.coef_

array([[4989.90118006, -117.54738915,  -71.5693016 , -811.65093156,
          23.86496768,  -22.98019608,  128.63802707, -551.41625636,
         835.50098553]])

In [18]:
regression.intercept_

array([3944.254662])

In [19]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [20]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1179.644591480174
MAE: 813.8092266859256
R2 score 91.44173860508148


Lasso
Model Training Performance
RMSE: 1180.8351588815315
MAE: 815.4198090362225
R2 score 91.42445487594959


Ridge
Model Training Performance
RMSE: 1179.8232038670917
MAE: 814.0750740594746
R2 score 91.43914676134864


Elasticnet
Model Training Performance
RMSE: 1629.2689629476954
MAE: 1079.23871584633
R2 score 83.6744149366052




In [21]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']